In [44]:
import os
from shutil import rmtree, copy
from glob import glob
from random import sample

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
DATA_MOD_DIR = DATA_DIR + '_mod'

def get_classes():
    return {
                i: os.path.split(name)[-1] for i, name in
                enumerate(sorted(glob(os.path.join(DATA_DIR, 'test', '*'))))
           }

def make_data_mod_uni(files_per_class, refresh):
    if os.path.exists(DATA_MOD_DIR) and not refresh:
        return

    if os.path.exists(DATA_MOD_DIR):
        rmtree(DATA_MOD_DIR)
    
    for name in get_classes().values():
        os.makedirs(os.path.join(DATA_MOD_DIR, name))
        files_to_copy = glob(os.path.join(DATA_DIR, 'train', name, '*'))
        files_to_copy = sample(files_to_copy, min(files_per_class, len(files_to_copy)))
        for f in files_to_copy:
            copy(f, os.path.join(DATA_MOD_DIR, name))
    return            

make_data_mod_uni(25, refresh=False)

In [1]:
import numpy as np
import tensorflow as tf

RAW_IMAGE_HEIGHT = 224
RAW_IMAGE_WIDTH = 224
RAW_IMAGE_CHANNELS = 3

batch_size = 256
dataset_train = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'train'),
    batch_size=batch_size,
    image_size=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH),
    crop_to_aspect_ratio=True,
    seed=1,
    validation_split=0.5,
    subset='training'    
)
dataset_valid = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'valid'),
    #label_mode='categorical',
    batch_size=batch_size,
    image_size=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH),
    crop_to_aspect_ratio=True
)

2023-09-23 17:03:48.979639: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 17:03:49.225600: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 17:03:49.228702: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 17:03:50.336736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 84635 files belonging to 525 classes.
Using 42318 files for training.
Found 2626 files belonging to 525 classes.


In [ ]:
pretrained_model = tf.keras.applications.efficientnet.EfficientNetB0(
    input_shape=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH, RAW_IMAGE_CHANNELS),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

pretrained_model.trainable = False

#checkpoint_path = 'birds_classification_model_checkpoint'
#checkpoint_path = 'birds_checkpoint_256'
checkpoint_path = 'birds_checkpoint'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor="val_loss",
    save_best_only=True
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",
    patience = 5,
    restore_best_weights = True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

augment = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),
])

inputs = pretrained_model.input
x = augment(inputs)

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dropout(0.45)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.45)(x)

outputs = tf.keras.layers.Dense(525, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

train_history = model.fit(
    dataset_train,
    steps_per_epoch=len(dataset_train),
    validation_data=dataset_valid,
    validation_steps=len(dataset_valid),
    epochs=50,
    callbacks=[
        early_stopping,
        checkpoint_callback,
        reduce_lr
    ]
)

Epoch 1/50
166/166 [==============================] - ETA: 0s - loss: 6.5764 - accuracy: 0.0030 INFO:tensorflow:Assets written to: birds_checkpoint/assets


INFO:tensorflow:Assets written to: birds_checkpoint/assets


166/166 [==============================] - 2197s 13s/step - loss: 6.5764 - accuracy: 0.0030 - val_loss: 6.2371 - val_accuracy: 0.0038 - lr: 1.0000e-04
Epoch 2/50
166/166 [==============================] - ETA: 0s - loss: 6.2381 - accuracy: 0.0040 INFO:tensorflow:Assets written to: birds_checkpoint/assets


INFO:tensorflow:Assets written to: birds_checkpoint/assets


166/166 [==============================] - 1956s 12s/step - loss: 6.2381 - accuracy: 0.0040 - val_loss: 6.1745 - val_accuracy: 0.0095 - lr: 1.0000e-04
Epoch 3/50
166/166 [==============================] - ETA: 0s - loss: 6.1599 - accuracy: 0.0090 INFO:tensorflow:Assets written to: birds_checkpoint/assets


...

166/166 [==============================] - 1870s 11s/step - loss: 1.8989 - accuracy: 0.5069 - val_loss: 0.6079 - val_accuracy: 0.8736 - lr: 1.0000e-04
Epoch 41/50
166/166 [==============================] - ETA: 0s - loss: 1.8723 - accuracy: 0.5123 INFO:tensorflow:Assets written to: birds_checkpoint/assets


INFO:tensorflow:Assets written to: birds_checkpoint/assets


166/166 [==============================] - 1844s 11s/step - loss: 1.8723 - accuracy: 0.5123 - val_loss: 0.6015 - val_accuracy: 0.8751 - lr: 1.0000e-04
Epoch 42/50
166/166 [==============================] - ETA: 0s - loss: 1.8373 - accuracy: 0.5248 INFO:tensorflow:Assets written to: birds_checkpoint/assets


INFO:tensorflow:Assets written to: birds_checkpoint/assets


166/166 [==============================] - 1828s 11s/step - loss: 1.8373 - accuracy: 0.5248 - val_loss: 0.5905 - val_accuracy: 0.8766 - lr: 1.0000e-04
Epoch 43/50
154/166 [==========================>...] - ETA: 2:01 - loss: 1.8015 - accuracy: 0.5311

In [2]:
from skimage.io import imread

checkpoint_path = 'birds_checkpoint'
model_1 = tf.keras.models.load_model(checkpoint_path)
dataset_valid.class_names[np.argmax(model_1.predict(np.array([imread('data/test/PHILIPPINE EAGLE/1.jpg')])))]

1/1 [==============================] - 1s 1s/step


'PHILIPPINE EAGLE'

In [3]:
model_1.evaluate(dataset_valid)

2023-09-23 17:21:15.470191: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-09-23 17:21:15.739765: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-09-23 17:21:15.995590: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-09-23 17:21:16.307256: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-09-23 17:21:16.488836: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


11/11 [==============================] - 149s 13s/step - loss: 0.5905 - accuracy: 0.8766


[0.5904751420021057, 0.8766184449195862]

In [9]:
len(y), y_pred.shape

(2626, (2626, 525))

In [10]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

predictions = model_1.predict(dataset_valid)

#y = np.argmax([l for _, ls in dataset_valid for l in ls], axis=1)
#y_pred = np.argmax(predictions, axis=1)
y = [l for _, ls in dataset_valid for l in ls]
y_pred = np.argmax(predictions, axis=1)

print(f1_score(y, y_pred, average='weighted'))
print(accuracy_score(y, y_pred))

11/11 [==============================] - 123s 11s/step
0.0022425319455022428
0.002284843869002285


In [6]:
predictions.shape

(2626, 525)

In [ ]:
print(len(y), len(y_pred))
y, y_pred

In [32]:
import tensorflow as tf

precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

def F1_score(y_true, y_pred):
    precision.update_state(y_true, y_pred)
    recall.update_state(y_true, y_pred)
    precision_result = precision.result()
    recall_result = recall.result()
    return 2 * ((precision_result * recall_result) / (precision_result + recall_result + 1e-6))

tf.keras.utils.get_custom_objects()['F1_score'] = F1_score

model_2 = tf.keras.models.load_model('data/EfficientNetB0-525-(224 X 224)- 98.97.h5', custom_objects={'F1_score':'F1_score'})
#model_2 = tf.keras.models.load_model('data/EfficientNetB0-525-(224 X 224)- 98.97.h5')
model_2.trainable = False
print(model_2)
dataset_valid.class_names[np.argmax(model_2.predict(np.array([imread('data/test/PHILIPPINE EAGLE/1.jpg')])))]

1/1 [==============================] - 1s 1s/step


'PHILIPPINE EAGLE'

In [33]:
model_2.evaluate(dataset_valid)

11/11 [==============================] - 83s 7s/step - loss: 0.3303 - accuracy: 0.9794 - F1_score: 0.9776


[0.33025839924812317, 0.9794363975524902, 0.9775663614273071]

In [ ]:
for i, j in zip(dataset_train.class_names, dataset_valid.class_names):
    if i != j:
        print(i, j)

In [51]:
dataset_mod_train = tf.keras.utils.image_dataset_from_directory(
    DATA_MOD_DIR,
    batch_size=batch_size,
    image_size=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH),
    crop_to_aspect_ratio=True,
    #seed=1  
)
dataset_valid = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'valid'),
    #label_mode='categorical',
    batch_size=batch_size,
    image_size=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH),
    crop_to_aspect_ratio=True
)

Found 13125 files belonging to 525 classes.
Found 2626 files belonging to 525 classes.


In [52]:
import keras_tuner as kt

def model_builder(hp):

    optim_dict = {1: tf.keras.optimizers.Adam, 2: tf.keras.optimizers.AdamW}
    optim_str_dict = {1: 'adam', 2: 'adamw'}
    optim_int = hp.Choice('optim_int', values=[1, 2])
    optim = optim_dict[optim_int]
    init_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    params = {'optim': optim_str_dict[optim_int], 'init_lr': init_lr}

    checkpoint_path = os.path.join('checkpoints', '_'.join([f'{k}-{v}' for k, v in params.items()]))
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor="val_loss",
        save_best_only=True
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",
        patience = 5,
        restore_best_weights = True
    )

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

    pretrained_model = tf.keras.applications.efficientnet.EfficientNetB0(
        input_shape=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH, RAW_IMAGE_CHANNELS),
        include_top=False,
        weights='imagenet',
        pooling='max'
    )
    pretrained_model.trainable = False
    
    augment = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),
    ])
    
    inputs = pretrained_model.input
    x = augment(inputs)
    
    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dropout(0.45)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.45)(x)
    
    outputs = tf.keras.layers.Dense(525, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=optim(init_lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = kt.Hyperband(model_builder,
     objective='val_accuracy',
     max_epochs=10,
     factor=3,
     directory='my_dir',
     project_name='intro_to_kt'
)

tuner.search(*dataset_mod_train, epochs=5, validation_data=dataset_valid)
best_model = tuner.get_best_models()[0]

Trial 2 Complete [00h 00m 03s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 09s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
2                 |2                 |optim_int
0.001             |0.0001            |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round



Traceback (most recent call last):
  File "/home/jmlazaro/.local/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/jmlazaro/.local/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jmlazaro/.local/lib/python3.11/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jmlazaro/.local/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/jmlazaro/.local/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/jmlazaro/.local/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jmlazaro/.local/lib/python3.11/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jmlazaro/.local/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
         